In [1]:
#I'll point to a file
#I'll parse out the file name and get the start and end date

#I'll parse out the int16 percision

#the output will be a dataframe with the timestamp for each sample and the float

#check that the last timestamp is the same as the end date

#graph the data

#install the wvunpack tool
#sudo apt-get install wvunpack

In [26]:
descriptor = "c57d828b-e8d1-433b-ad79-5420d2136d3f_i2c-0-0x4b_bosch-bno085_acceleration_mDsE2_int16-f6_1_32hz"
file_name = "20251220T000000p000Z_20251220T235959p968Z.wv"
num_channels = 3

# descriptor = "c57d828b-e8d1-433b-ad79-5420d2136d3f_i2c-0-0x4b_bosch-bno085_game-rotation_quaternion_int16-f10_1_16hz"
# file_name = "20251219T171251p000Z_20251219T224253p312Z.wv"
# num_channels = 4

# descriptor = "c57d828b-e8d1-433b-ad79-5420d2136d3f_i2c-0-0x4b_bosch-bno085_gyroscope_radDs_int16-f6_1_32hz"
# descriptor = "c57d828b-e8d1-433b-ad79-5420d2136d3f_i2c-0-0x4b_bosch-bno085_magnetometer_ut_int16-f6_1_8hz"
# descriptor = "c57d828b-e8d1-433b-ad79-5420d2136d3f_i2c-1-0x77_bosch-bme680_air-temperature_celsius_int16-f8_1_1hz"
# descriptor = "c57d828b-e8d1-433b-ad79-5420d2136d3f_i2c-1-0x77_bosch-bme680_barometric-pressure_kpa_int16-f7_1_16hz"
# descriptor = "c57d828b-e8d1-433b-ad79-5420d2136d3f_i2c-1-0x77_bosch-bme680_relative-humidity_percent_int16-f8_1_0P25hz"
# descriptor = "c57d828b-e8d1-433b-ad79-5420d2136d3f_i2c-1-0x77_bosch-bme680_volatile-organic-compounds_LNohm_int16-f11_1_1hz"


In [27]:
from datetime import datetime, timezone

start_dt = datetime.strptime(file_name.split("_")[0], "%Y%m%dT%H%M%Sp%fZ").replace(tzinfo=timezone.utc)
end_dt = datetime.strptime(file_name.split("_")[1], "%Y%m%dT%H%M%Sp%fZ.wv").replace(tzinfo=timezone.utc)


interval_s = 1/float(descriptor.split("_")[-1][:-2])
data_type_descriptor = descriptor.split("_")[-3]
endianness = "le"


float_scale = 1
if "-" in data_type_descriptor:
    np_type = data_type_descriptor.split("-")[0]
    float_scale = 2 ** int(data_type_descriptor.split("-")[1][1:])
else:
    np_type = data_type_descriptor


file_base_path = "/home/chowder/Documents/testData/" + descriptor + "/" + start_dt.strftime("%Y/%m/%d/")
full_file_path = file_base_path + file_name

print(full_file_path)


/home/chowder/Documents/testData/c57d828b-e8d1-433b-ad79-5420d2136d3f_i2c-0-0x4b_bosch-bno085_acceleration_mDsE2_int16-f6_1_32hz/2025/12/20/20251220T000000p000Z_20251220T235959p968Z.wv


In [28]:
import subprocess
import numpy as np
import pandas as pd
import os

#check that the file exists
if not os.path.exists(full_file_path):
    raise FileNotFoundError(f"File not found: {full_file_path}")
else:
    print(f"File found: {full_file_path}")

# Decompress the wavpak file to raw PCM data
# Use --raw to get raw PCM output, and -o - to output to stdout
wvunpack_cmd = ["wvunpack", "--raw", full_file_path, "-o", "-"]
result = subprocess.run(wvunpack_cmd, capture_output=True, check=True)
raw_data = result.stdout

# Convert binary data to numpy array
# int16 little-endian: '<i2'
dtype = '<i2'  # int16 little-endian
data_array = np.frombuffer(raw_data, dtype=dtype)

File found: /home/chowder/Documents/testData/c57d828b-e8d1-433b-ad79-5420d2136d3f_i2c-0-0x4b_bosch-bno085_acceleration_mDsE2_int16-f6_1_32hz/2025/12/20/20251220T000000p000Z_20251220T235959p968Z.wv


In [29]:
colNames = ["1", "2", "3", "4"]

# Reshape to (n_samples, num_channels) for X, Y, Z
n_samples = len(data_array) // num_channels
data_reshaped = data_array[:n_samples * num_channels].reshape(n_samples, num_channels)

# Scale the data using float_scale
data_scaled = data_reshaped.astype(float) / float_scale

# Create timezone-aware datetime index
sampleDT = pd.date_range(start=start_dt, periods=n_samples, freq=pd.Timedelta(seconds=interval_s), tz='UTC')

# Create DataFrame with sampleDT as index and X, Y, Z as columns
df = pd.DataFrame(data_scaled, columns=colNames[:num_channels], index=sampleDT)
df.index.name = 'sampleDT'

print(f"DataFrame shape: {df.shape}")
print(f"Time range: {df.index[0]} to {df.index[-1]}")
print(f"\nFirst few rows:")
print(df.head())

#even for the more noisy data, it's looking like about 3 bits per number

DataFrame shape: (2764800, 3)
Time range: 2025-12-20 00:00:00+00:00 to 2025-12-20 23:59:59.968750+00:00

First few rows:
                                        1        2         3
sampleDT                                                    
2025-12-20 00:00:00+00:00         0.81250 -0.65625  9.578125
2025-12-20 00:00:00.031250+00:00  0.81250 -0.65625  9.578125
2025-12-20 00:00:00.062500+00:00  0.84375 -0.65625  9.578125
2025-12-20 00:00:00.093750+00:00  0.84375 -0.65625  9.546875
2025-12-20 00:00:00.125000+00:00  0.84375 -0.65625  9.546875


In [30]:

df



,1,2,3
sampleDT,,,
2025-12-20 00:00:00+00:00,0.81250,-0.65625,9.578125
2025-12-20 00:00:00.031250+00:00,0.81250,-0.65625,9.578125
2025-12-20 00:00:00.062500+00:00,0.84375,-0.65625,9.578125
2025-12-20 00:00:00.093750+00:00,0.84375,-0.65625,9.546875
2025-12-20 00:00:00.125000+00:00,0.84375,-0.65625,9.546875
...,...,...,...
2025-12-20 23:59:59.843750+00:00,0.81250,-0.65625,9.578125
2025-12-20 23:59:59.875000+00:00,0.81250,-0.68750,9.578125
2025-12-20 23:59:59.906250+00:00,0.81250,-0.68750,9.578125
